# Sistema de generación de canciones
## Bogdan Kaleb García Rivera 
## MIA-2

Lectura del archivo: 

In [1]:
import pandas as pd 
df = pd.read_csv("./songs_filtered.csv")
df.head()

,song_name,artist_name,album_name,lyrics,release_date,producers,tags,porcentaje_ruido,idioma,porcentaje_ruido_limpio,tokens
0,Ya ya,Hurriganes,Jailbird,"Sitting in the la Waiting for the ya, uh huh S...",NaN,[],['Pop'],3.112840,en,3.043478,"['Sitting', 'in', 'the', 'la', 'Waiting', 'for..."
1,You Already Know,Fergie,Double Dutchess (Target Exclusive),"<part> Life's a movie, let the camera roll Fa...","August 25, 2017",[],"['R&B', 'Rap', 'Pop', 'USA', 'In English', 'El...",5.072793,en,4.873387,"['<', 'part', '>', 'Life', ""'s"", 'a', 'movie',..."
2,Our Embrace,Uneven Structure,La Partition,"You processed me to bile, vulgar and featurele...",NaN,[],['Rock'],0.608696,en,0.537153,"['You', 'processed', 'me', 'to', 'bile', ',', ..."
3,Higher quiddity,Uneven Structure,NaN,Being tricked as a tool of anti-evolution A po...,NaN,[],['Pop'],1.129944,en,0.714286,"['Being', 'tricked', 'as', 'a', 'tool', 'of', ..."
4,Egocentric Focus,Uneven Structure,8 - EP,With that egocentric focus The unending whirl ...,NaN,[],['Pop'],1.184834,en,1.041667,"['With', 'that', 'egocentric', 'focus', 'The',..."


  Este proyecto únicamente se enfocará en canciones en inglés.

In [2]:
import re
df_english = df[df['idioma'] == 'en'].copy()
df_english.head()

,song_name,artist_name,album_name,lyrics,release_date,producers,tags,porcentaje_ruido,idioma,porcentaje_ruido_limpio,tokens
0,Ya ya,Hurriganes,Jailbird,"Sitting in the la Waiting for the ya, uh huh S...",NaN,[],['Pop'],3.112840,en,3.043478,"['Sitting', 'in', 'the', 'la', 'Waiting', 'for..."
1,You Already Know,Fergie,Double Dutchess (Target Exclusive),"<part> Life's a movie, let the camera roll Fa...","August 25, 2017",[],"['R&B', 'Rap', 'Pop', 'USA', 'In English', 'El...",5.072793,en,4.873387,"['<', 'part', '>', 'Life', ""'s"", 'a', 'movie',..."
2,Our Embrace,Uneven Structure,La Partition,"You processed me to bile, vulgar and featurele...",NaN,[],['Rock'],0.608696,en,0.537153,"['You', 'processed', 'me', 'to', 'bile', ',', ..."
3,Higher quiddity,Uneven Structure,NaN,Being tricked as a tool of anti-evolution A po...,NaN,[],['Pop'],1.129944,en,0.714286,"['Being', 'tricked', 'as', 'a', 'tool', 'of', ..."
4,Egocentric Focus,Uneven Structure,8 - EP,With that egocentric focus The unending whirl ...,NaN,[],['Pop'],1.184834,en,1.041667,"['With', 'that', 'egocentric', 'focus', 'The',..."


Generación de N-gramas: Inicialmente, el proceso de generación de n gramas consta primeramente de realizar al tokenización cada letra de canción. Esta tokenización se realiza a través de palabras, por lo que se pueden almacenar en un vector. Por ejemplo, "El gato duerme en el sillón" se puede tokenizar como: ['el','gato','duerme','en','el','sillón']. Posterior a eso se realiza el proceso de generación de n-gramas. El proceso es tomar el número requerido de n gramas e ir guardándolo en una tupla y posteriormente en un arreglo. Por ejemplo, del ejemplo anterior para n=2 se puede separar como: 

[('el', 'gato'), ('gato', 'duerme'), ('duerme', 'en'), ('en', 'el'), ('el', 'sillón')]

Finalmente se cuentan cuantos n_gramas parecidos existen en el corpus. 

In [ ]:
from collections import defaultdict
import pandas as pd

def tokenize(text):
    return text.lower().split()

def generate_ngrams(tokens, n):
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]

def count_ngrams(text_series, n):
    ngram_counts = defaultdict(int)

    for text in text_series.dropna():
        tokens = tokenize(text)
        ngrams = generate_ngrams(tokens, n)
        for ngram in ngrams:
            ngram_counts[ngram] += 1

    return ngram_counts

def get_top_ngrams(ngram_counts, top_k=10):
    return sorted(ngram_counts.items(), key=lambda x: x[1], reverse=True)[:top_k]

def print_top_ngrams(ngram_counts, top_k=10):
    top_ngrams = get_top_ngrams(ngram_counts, top_k)
    for ngram, count in top_ngrams:
        print(f"{ngram}: {count} apariciones")


n = 3  
ngram_counts = count_ngrams(df_english['lyrics'], n)
print_top_ngrams(ngram_counts, top_k=10)


. . .: 828 apariciones
i don't know: 553 apariciones
i don't wanna: 530 apariciones
i got a: 454 apariciones
i know you: 415 apariciones
i know that: 413 apariciones
up in the: 356 apariciones
i just wanna: 354 apariciones
you know that: 335 apariciones
i don't want: 319 apariciones


A partir del código previo, es posible comenzar a construir el modelo de n-gramas. El enfoque sigue la misma lógica que antes, pero para realizar el conteo es necesario dividir el texto en prefijo y palabra siguiente (next_word). En cada frase, se extraen los n-gramas con n-1 palabras como prefijo, mientras que la palabra siguiente se gestiona en un vector separado para facilitar la predicción, basado en la cantidad de ocurrencias de ese n-grama con dicha palabra.
Asimismo, se calculan las probabilidades individuales y acumuladas para cada combinación de prefijo y next_word. Por ejemplo, en el caso de "s sitting", las palabras siguientes y sus respectivas probabilidades son:

s sitting:

    'in': probabilidad = 36.36%, acumulada = 36.36%

    'alone': probabilidad = 18.18%, acumulada = 54.55%

    'on': probabilidad = 18.18%, acumulada = 72.73%

    'at': probabilidad = 9.09%, acumulada = 81.82%

    'where': probabilidad = 9.09%, acumulada = 90.91%

    'around': probabilidad = 9.09%, acumulada = 100.00%


In [4]:
import pandas as pd
import re
import random
from collections import defaultdict

def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

def generate_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]


def ngram_model(text_series, n):
    prefix_counts = defaultdict(int) # Cuenta los prefijos 
    model = defaultdict(lambda: defaultdict(int))  # Se crea un diccionario de diccionarios 

    for text in text_series.dropna(): #Quitar valores na 
        tokens = tokenize(text)
        ngrams = generate_ngrams(tokens, n)
        for gram in ngrams:
            prefix = ' '.join(gram[:-1]) #No toma en cuenta la última palabra 
            next_word = gram[-1] #Palabra final 
            model[prefix][next_word] += 1 #Se cuenta cuantas veces aparece la combinación 
            prefix_counts[prefix] += 1

    # Conversión de conteos a probabilidades acumuladas
    probs = {}
    for prefix, next_words in model.items():
        total = sum(next_words.values())
        sorted_words = sorted(next_words.items(), key=lambda x: -x[1])  # Ordenar de mayor a menor
        p_acumulada = 0
        probs[prefix] = []
        for word, count in sorted_words:
            prob = count / total
            p_acumulada += prob
            probs[prefix].append((word, prob, p_acumulada))

    return probs

Para determinar la siguiente palabra, simplemente se genera un número aleatorio entre 0 y 1 y se verifica en qué intervalo de la frecuencia acumulada encaja, tomando en cuenta las palabras asociadas al n-grama - 1 correspondiente.

In [5]:
def predict_next_word(prefix, probs):
    if prefix not in probs:
        return None  # No existe la palabra en el corpus 
    r = random.random()
    for word, prob, p_acumulada in probs[prefix]:
        if r < p_acumulada:
            return word
    return probs[prefix][-1][0] 

Haciendo pruebas de predicción de palabras con 3 n-gramas: 

In [6]:
n = 3
probs = ngram_model(df_english['lyrics'], n)
prefix = "i love"
predicted = predict_next_word(prefix, probs)
print(f"Siguiente palabra de '{prefix}': {predicted}")

Siguiente palabra de 'i love': you


In [7]:
for prefix in list(probs.keys())[:10]:  # Muestra los primeros 10 n-gramas con sus probabilidades
    print(f"{prefix}: {probs[prefix]}")

sitting in: [('the', 0.21052631578947367, 0.21052631578947367), ('my', 0.19298245614035087, 0.40350877192982454), ('a', 0.17543859649122806, 0.5789473684210527), ('your', 0.15789473684210525, 0.736842105263158), ('front', 0.07017543859649122, 0.8070175438596492), ('this', 0.03508771929824561, 0.8421052631578948), ('his', 0.03508771929824561, 0.8771929824561404), ('church', 0.017543859649122806, 0.8947368421052633), ('girl', 0.017543859649122806, 0.9122807017543861), ('it', 0.017543859649122806, 0.929824561403509), ('two', 0.017543859649122806, 0.9473684210526319), ('handcuffs', 0.017543859649122806, 0.9649122807017547), ('court', 0.017543859649122806, 0.9824561403508776), ('an', 0.017543859649122806, 1.0000000000000004)]
in the: [('morning', 0.02642736009044658, 0.02642736009044658), ('back', 0.02642736009044658, 0.05285472018089316), ('air', 0.023318258903335217, 0.07617297908422838), ('sky', 0.022611644997173545, 0.09878462408140193), ('dark', 0.021339739966082533, 0.1201243640474844

Para generar frases, se aplica el mismo principio que en la predicción de palabras. En esta función, el prefijo debe contener al menos n-1 palabras; si no cumple con este requisito, se descarta. Si la siguiente palabra dentro del conjunto de n-1 no está disponible, el proceso se interrumpe. Finalmente, las palabras generadas se combinan dentro de una ventana de tamaño n.
Por ejemplo, con un n-grama de tamaño 2 y una longitud de 6 palabras, se tiene lo siguiente: 

probs = {

    "el gato": [("duerme", 0.5, 0.5), ("juega", 0.3, 0.8), ("salta", 0.2, 1.0)],

    "gato duerme": [("tranquilo", 1.0, 1.0)],

    "gato juega": [("con", 0.7, 0.7), ("solo", 0.3, 1.0)],

}


Las posibles respuestas serían: 

el gato juega con el perro

el gato duerme tranquilo

el gato salta rápidamente



In [8]:
def generar_frase(probs, n=3, longitud=5):
    posibles_prefijos = list(probs.keys())
    prefijo = random.choice(posibles_prefijos) #Se elige aleatoriamente un prefijo
    palabras = prefijo.split() 

    # Si el prefijo tiene menos de n-1 palabras, se descarta
    if len(palabras) != n - 1:
        return generar_frase(probs, n, longitud)

    while len(palabras) < longitud:
        contexto = ' '.join(palabras[-(n-1):])  # mantener ventana de n-1
        siguiente = predict_next_word(contexto, probs)
        if siguiente is None:
            break  # No hay continuación
        palabras.append(siguiente)

    return ' '.join(palabras)

Se puede construir frases de trigramas: 

In [9]:
# Construir el modelo
probs = ngram_model(df_english['lyrics'], n=3)

# Generando 5 frases
for _ in range(5):
    print(generar_frase(probs, n=3, longitud=100))

cried joseph hey god spoke to god i know verse i want you so i sit and wait for it get ready for it and you too animated gimmicks nothin new under the moonlight sandy beaches drinking rum every night when sleep is hard to succeed when ya finally do this on my neck is bent and dented in with the connections hunh who s casually throwing some firecrackers around rahad is dancing on your wagon verse the red mist is a dog this is punishment i ll be gone then repeat i don t put here to help me
in axe shah s face cut to 116 int miriam s a wattage problem yeah i just popped a stash at his back red tips rippin through his fleece and rang all through a thin sheen of sweat covering his crotch shakes her head into your eyes lookin at your booth i m in philly i top rock b brand new matte black aventador same color shades shade earn your place lately the lights off yes with the gun in the sea she roars with laughter and the road made sure i m not fazed deep thoughts on what we
losing your will till 

Para calcular la perplejidad se hace uso de la siguiente fórmula: 

In [10]:
def train_ngram_counts(text_series, n=3):
    ngram_counts = defaultdict(int)
    context_counts = defaultdict(int)
    
    for text in text_series.dropna():
        tokens = tokenize(text)
        tokens = ['<s>'] * (n-1) + tokens + ['</s>']
        
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            context = tuple(tokens[i:i+n-1])
            ngram_counts[ngram] += 1
            context_counts[context] += 1
    
    return ngram_counts, context_counts

Dividir 80% de entrenamiento y prueba y generación de n-gramas: 

In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df_english['lyrics'], test_size=0.2,random_state=50)

n = 3  # Trigramas
ngram_counts, context_counts = train_ngram_counts(train_data, n)


In [12]:
ngram_counts,context_counts = train_ngram_counts(train_data,3)
ngram_counts

defaultdict(int,
            {('<s>', '<s>', 'hiss'): 1,
             ('<s>', 'hiss', 'then'): 1,
             ('hiss', 'then', 'a'): 1,
             ('then', 'a', 'cluster'): 1,
             ('a', 'cluster', 'of'): 1,
             ('cluster', 'of', 'tones'): 1,
             ('of', 'tones', 'slurring'): 1,
             ('tones', 'slurring', 'through'): 1,
             ('slurring', 'through', 'haze'): 1,
             ('through', 'haze', 'luring'): 1,
             ('haze', 'luring', 'me'): 1,
             ('luring', 'me', 'to'): 1,
             ('me', 'to', 'go'): 22,
             ('to', 'go', 'forth'): 1,
             ('go', 'forth', 'to'): 1,
             ('forth', 'to', 'bring'): 1,
             ('to', 'bring', 'them'): 1,
             ('bring', 'them', 'back'): 1,
             ('them', 'back', 'to'): 5,
             ('back', 'to', 'the'): 147,
             ('to', 'the', 'place'): 14,
             ('the', 'place', 'where'): 32,
             ('place', 'where', 'they'): 3,
             

In [13]:
context_counts

defaultdict(int,
            {('<s>', '<s>'): 4191,
             ('<s>', 'hiss'): 1,
             ('hiss', 'then'): 1,
             ('then', 'a'): 22,
             ('a', 'cluster'): 1,
             ('cluster', 'of'): 1,
             ('of', 'tones'): 1,
             ('tones', 'slurring'): 1,
             ('slurring', 'through'): 1,
             ('through', 'haze'): 1,
             ('haze', 'luring'): 1,
             ('luring', 'me'): 1,
             ('me', 'to'): 610,
             ('to', 'go'): 535,
             ('go', 'forth'): 3,
             ('forth', 'to'): 5,
             ('to', 'bring'): 50,
             ('bring', 'them'): 10,
             ('them', 'back'): 17,
             ('back', 'to'): 541,
             ('to', 'the'): 2540,
             ('the', 'place'): 100,
             ('place', 'where'): 71,
             ('where', 'they'): 62,
             ('they', 'belong'): 5,
             ('belong', 'close'): 1,
             ('close', 'to'): 103,
             ('the', 'eight'): 5,
      

In [14]:
print("Train data shape: ",train_data.shape)
print("Test data shape: ",test_data.shape)

Train data shape:  (4191,)
Test data shape:  (1048,)


### Perplexity

In [15]:
def get_prob(ngram):
    context = ngram[:-1]
    return ngram_counts[ngram] / context_counts[context] if context_counts[context] > 0 else 0.0

Probando get_prob(): 

In [16]:
frase = "back to the place"
 
tokens = ['<s>', '<s>'] + tokenize(frase) + ['</s>']
n = 3
test_ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
for ngram in test_ngrams:
    prob = get_prob(ngram)
    print(f"{ngram} -> {prob}")


('<s>', '<s>', 'back') -> 0.0
('<s>', 'back', 'to') -> 0.0
('back', 'to', 'the') -> 0.27171903881700554
('to', 'the', 'place') -> 0.005511811023622047
('the', 'place', '</s>') -> 0.0


In [17]:
import math
def perplexity(test_sentence, n):
    test_tokens = ['<s>'] * (n-1) + test_sentence.lower().split() + ['</s>']
    N = len(test_tokens) - n + 1
    log_prob = 0
    for i in range(N):
        ngram = tuple(test_tokens[i:i+n])
        prob = get_prob(ngram)
        if prob > 0:
            log_prob += math.log(prob)
        else:
            # Evita log(0) y penaliza fuertemente
            return float('inf')
    return math.exp(-log_prob / N)

In [18]:
def evaluate_model(test_data, n):
    total_perplexity = 0.0
    valid_sentences = 0
    
    for sentence in test_data.dropna():
        if isinstance(sentence, str):
            pp = perplexity(sentence, n)
            if not math.isinf(pp):
                total_perplexity += pp
                valid_sentences += 1
    
    if valid_sentences == 0:
        return float('inf')
    
    prom_per = total_perplexity / valid_sentences
    print(f"Evaluado en {valid_sentences} frases válidas de {len(test_data)} totales")
    print(f"Perplexity promedio: {prom_per:.2f}")
    return prom_per

print("Entrenando modelo...")
model_probs = ngram_model(train_data, n=3)  # Trigramas

print("\nGenerando frases de ejemplo:")
for _ in range(2):
    print(generar_frase(model_probs, n=3, longitud=10))

print("\nEvaluación con test")
prom_per = evaluate_model(test_data, n=3)
if prom_per == float('inf'): 
    print("Promedio: ",prom_per)


Entrenando modelo...



Generando frases de ejemplo:
o dibala jwang letšatši la gago le jwang oh yes
all break your poo la means i can t replace

Evaluación con test
Promedio:  inf


Para el caso de que la perplexity diera 'inf' significa que no existe ninguna frase en el corpus que coincida directamente con el test. 

### Suavizado de Laplace 

In [19]:
# Construcción del vocabulario desde los datos de entrenamiento
vocab = set()
for sentence in train_data.dropna():
    tokens = tokenize(sentence)
    vocab.update(tokens)

vocab_size = len(vocab) + 1  # +1 por el token </s>
print(f"Tamaño del vocabulario: {vocab_size}")

Tamaño del vocabulario: 47646


In [20]:
def get_prob_laplace(ngram):
    context = ngram[:-1]
    return (ngram_counts[ngram] + 1) / (context_counts[context] + vocab_size)

Probando get_prob_laplace(): 

In [21]:
frase = "back to the place"
 
tokens = ['<s>', '<s>'] + tokenize(frase) + ['</s>']
n = 3
test_ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
for ngram in test_ngrams:
    prob = get_prob_laplace(ngram)
    print(f"{ngram} -> {prob}")


('<s>', '<s>', 'back') -> 1.929123984798503e-05
('<s>', 'back', 'to') -> 2.0988120723670403e-05
('back', 'to', 'the') -> 0.0030713677962936063
('to', 'the', 'place') -> 0.00029888813613358306
('the', 'place', '</s>') -> 2.0944162861810414e-05


In [22]:
def perplexity_laplace(test_sentence, n):
    test_tokens = ['<s>'] * (n-1) + test_sentence.lower().split() + ['</s>']
    N = len(test_tokens) - n + 1
    log_prob = 0
    for i in range(N):
        ngram = tuple(test_tokens[i:i+n])
        prob = get_prob_laplace(ngram)
        log_prob += math.log(prob)
    return math.exp(-log_prob / N)

In [23]:
def evaluate_model_laplace(test_data, n):
    total_perplexity = 0.0
    valid_sentences = 0
    
    for sentence in test_data.dropna():
        if isinstance(sentence, str):
            pp = perplexity_laplace(sentence, n)
            total_perplexity += pp
            valid_sentences += 1

    if valid_sentences == 0:
        return float('inf')
    
    prom_per = total_perplexity / valid_sentences
    print(f"Evaluado en {valid_sentences} frases válidas de {len(test_data)} totales")
    print(f"Perplexity promedio con Laplace: {prom_per:.2f}")
    return prom_per


print("Entrenando modelo...")
model_probs = ngram_model(train_data, n=3)

print("\nEvaluación con test (Laplace)")
prom_per = evaluate_model_laplace(test_data, n=3)

Entrenando modelo...



Evaluación con test (Laplace)
Evaluado en 1048 frases válidas de 1048 totales
Perplexity promedio con Laplace: 30459.90


El promedio de perplexity con Laplace nos indica un alto valor, por lo que podemos inferir que las letras tanto del conjunto de datos de prueba como el de entrenamiento no tienen un patrón tan definido. 

### Add-k smoothing

In [24]:
# Función de probabilidad con add-k smoothing
def get_prob_add_k(ngram, k=0.01):
    context = ngram[:-1]
    return (ngram_counts[ngram] + k) / (context_counts[context] + k * vocab_size)

In [25]:
# Perplexity con add-k
def perplexity_add_k(test_sentence, n, k=0.01):
    test_tokens = ['<s>'] * (n-1) + test_sentence.lower().split() + ['</s>']
    N = len(test_tokens) - n + 1
    log_prob = 0
    for i in range(N):
        ngram = tuple(test_tokens[i:i+n])
        prob = get_prob_add_k(ngram, k)
        log_prob += math.log(prob)
    return math.exp(-log_prob / N)

In [26]:
def evaluate_model_add_k(test_data, n, k=0.01, vocab_size=None):    
    total_perplexity = 0.0
    valid_sentences = 0
    unknown_ngrams = 0
    total_ngrams = 0
    
    for sentence in test_data.dropna():
        if isinstance(sentence, str):
            test_tokens = ['<s>'] * (n-1) + tokenize(sentence) + ['</s>']
            N = len(test_tokens) - n + 1
            total_ngrams += N
            
            if N <= 0:
                continue
                
            log_prob = 0.0
            current_unknown = 0
            
            for i in range(N):
                ngram = tuple(test_tokens[i:i+n])
                prob = get_prob_add_k(ngram, k)
                
                if prob == 0:
                    current_unknown += 1
                log_prob += math.log(prob) if prob > 0 else -math.inf
            
            unknown_ngrams += current_unknown
            
            # Solo se consideran frases con al menos 50% de n-gramas conocidos
            if current_unknown / N < 0.5:
                perplexity = math.exp(-log_prob / N) if not math.isinf(log_prob) else float('inf')
                total_perplexity += perplexity
                valid_sentences += 1
    
    # Resultados
    print(f"\nEvaluación con Add-k (k={k})")
    print(f"Frases válidas: {valid_sentences}/{len(test_data.dropna())}")
    print(f"N-gramas desconocidos: {unknown_ngrams}/{total_ngrams} ({unknown_ngrams/total_ngrams:.1%})")
    
    if valid_sentences == 0:
        avg_perplexity = float('inf')
    else:
        avg_perplexity = total_perplexity / valid_sentences
    
    return avg_perplexity

print("Entrenando modelo...")
model_probs = ngram_model(train_data, n=3)

print("\nEvaluación con Add-k smoothing:")
for k in [0.001, 0.01, 0.5, 1.0]:  # Diferentes valores de k 
    pp = evaluate_model_add_k(test_data, n=3, k=k, vocab_size=vocab_size)
    print(f"k={k}: Perplexity={pp:.2f}")

Entrenando modelo...

Evaluación con Add-k smoothing:

Evaluación con Add-k (k=0.001)
Frases válidas: 1048/1048
N-gramas desconocidos: 0/409739 (0.0%)
k=0.001: Perplexity=6399.11

Evaluación con Add-k (k=0.01)
Frases válidas: 1048/1048
N-gramas desconocidos: 0/409739 (0.0%)
k=0.01: Perplexity=6936.07

Evaluación con Add-k (k=0.5)
Frases válidas: 1048/1048
N-gramas desconocidos: 0/409739 (0.0%)
k=0.5: Perplexity=16061.22

Evaluación con Add-k (k=1.0)
Frases válidas: 1048/1048
N-gramas desconocidos: 0/409739 (0.0%)
k=1.0: Perplexity=19324.98


### Conclusión

Se puede notar que, en este corpus de canciones, la distribución de las letras es bastante dispersa tanto en el conjunto de entrenamiento como en el de prueba. Como resultado, los valores de Perplejidad son elevados, mostrando la gran variabilidad en las formas y temáticas de las canciones.